<a href="https://colab.research.google.com/github/ThOpaque/Food_Recognition/blob/main/YOLO_MNIST_Localization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, time,datetime
from timeit import default_timer as timer
import PIL.Image, PIL.ImageFont, PIL.ImageDraw
import numpy as np
from matplotlib import pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
%pip install torchinfo;
%pip install torchmetrics;
from torchmetrics import MeanSquaredError;
from torchinfo import summary;

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 419 kB 5.1 MB/s 


In [2]:
im_width = 75
im_height = 75
use_normalized_coordinates = True

def draw_ONE_bounding_box_on_image(image, ymin:int, xmin:int, ymax:int, xmax:int, 
                               color:str='red', thickness:int=1, display_str:bool=None, 
                               use_normalized_coordinates:bool=True):
  """Adds a bounding box to an image.
  Bounding box coordinates can be specified in either absolute (pixel) or
  normalized coordinates by setting the use_normalized_coordinates argument.
  
  Args:
    image: a PIL.Image object.
    ymin: ymin of bounding box.
    xmin: xmin of bounding box.
    ymax: ymax of bounding box.
    xmax: xmax of bounding box.
    color: color to draw bounding box. Default is red.
    thickness: line thickness. Default value is 4.
    display_str_list: string to display in box
    use_normalized_coordinates: If True (default), treat coordinates
      ymin, xmin, ymax, xmax as relative to the image.  Otherwise treat
      coordinates as absolute.
  """
  draw = PIL.ImageDraw.Draw(image)
  im_width, im_height = image.size
  
  if use_normalized_coordinates:
    (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                  ymin * im_height, ymax * im_height)
  else:
    (left, right, top, bottom) = (xmin, xmax, ymin, ymax)
  draw.line([(left, top), (left, bottom), (right, bottom),
             (right, top), (left, top)], width=thickness, fill=color)



def draw_bounding_boxes_on_image(image, boxes:np.ndarray, color:list=[], 
                                 thickness:int=1, display_str_list:tuple=()):
  """Draws bounding boxes on image.

  Args:
    image: a PIL.Image object.
    boxes: a 2 dimensional numpy array of [N, 4]: (ymin, xmin, ymax, xmax).
           The coordinates are in normalized format between [0, 1].
    color: color to draw bounding box. Default is red.
    thickness: line thickness. Default value is 4.
    display_str_list: a list of strings for each bounding box.
                           
  Raises:
    ValueError: if boxes is not a [N, 4] array
  """
  boxes_shape = boxes.shape
  if not boxes_shape:
    return
  if len(boxes_shape) != 2 or boxes_shape[1] != 4:
    raise ValueError('Input must be of size [N, 4]')
  
  for i in range(boxes_shape[0]):
    draw_ONE_bounding_box_on_image(image, 
                                   boxes[i, 1], boxes[i, 0], 
                                   boxes[i, 3], boxes[i, 2], 
                                   color[i], thickness, display_str_list[i])


def draw_bounding_boxes_on_image_array(image:np.ndarray, boxes:np.ndarray, color:list=[], 
                                       thickness:int=1, display_str_list:tuple=()):
  """Draws bounding boxes on image (numpy array).

  Args:
    image: a numpy array object.
    boxes: a 2 dimensional numpy array of [N, 4]: (ymin, xmin, ymax, xmax).
           The coordinates are in normalized format between [0, 1].
    color: color to draw bounding box. Default is red.
    thickness: line thickness. Default value is 4.
    display_str_list_list: a list of strings for each bounding box.
  Raises:
    ValueError: if boxes is not a [N, 4] array
  """
  image_pil = PIL.Image.fromarray(image)
  rgbimg = PIL.Image.new("RGBA", image_pil.size)
  rgbimg.paste(image_pil)
  draw_bounding_boxes_on_image(rgbimg, boxes, color, thickness, display_str_list)
  return np.array(rgbimg)

In [3]:
############### Matplotlib config
plt.rc('image', cmap='gray')
plt.rc('grid', linewidth=0)
plt.rc('xtick', top=False, bottom=False, labelsize='large')
plt.rc('ytick', left=False, right=False, labelsize='large')
plt.rc('axes', facecolor='F8F8F8', titlesize="large", edgecolor='white')
plt.rc('text', color='a8151a')
plt.rc('figure', facecolor='F0F0F0')# Matplotlib fonts
MATPLOTLIB_FONT_DIR = os.path.join(os.path.dirname(plt.__file__), "mpl-data/fonts/ttf")
################################################################################

In [4]:
################################################################################
def display_digits_with_boxes(digits, predictions, labels, pred_bboxes, bboxes, iou, title):
  """Utility to display a row of digits with their predictions.

  Args:
    digits : np.ndarray of shape (N,75,75,1)
        Raw image with normalized pixel values (from 0 to 1)
    predictions : np.ndarray of shape (N,)
        Predicted label with the same shape as labels
    labels : np.ndarray of shape (N,)
        Labels of the digits (from 0 to 9)
    pred_bboxes : np.ndarray of shape (n, N) ??
        Predicted bboxes locations
    bboxes : np.ndarray of shape (n, N)
        Ground true bboxe locations
    iou : list of shape (???)
        IoU of each bboxes
    title : str
        Figure's title
  """
  n = 10
  indexes = np.random.choice(len(predictions), size=n)
  n_digits = digits[indexes]
  n_predictions = predictions[indexes]
  n_labels = labels[indexes]

  n_iou = []
  if len(iou) > 0:
    # If multiple bboxes
    n_iou = iou[indexes]

  if (len(pred_bboxes) > 0):
    # If multiple bboxes predicted
    n_pred_bboxes = pred_bboxes[indexes,:]

  if (len(bboxes) > 0):
    # If multiple ground truth bboxes
    n_bboxes = bboxes[indexes,:]

  # Rescale pixel values to un-normed values (from 0 -black- to 255 -white-)
  n_digits = n_digits * 255.0
  n_digits = n_digits.reshape(n, 75, 75)

  # Set plot config
  fig = plt.figure(figsize=(20, 4))
  plt.title(title)
  plt.yticks([])
  plt.xticks([])
  
  for i in range(10):
    ax = fig.add_subplot(1, 10, i+1)
    bboxes_to_plot = []
    if (len(pred_bboxes) > i):
      bboxes_to_plot.append(n_pred_bboxes[i])
    
    if (len(bboxes) > i):
      bboxes_to_plot.append(n_bboxes[i])

    img_to_draw = draw_bounding_boxes_on_image_array(image=n_digits[i], boxes=np.asarray(bboxes_to_plot), color=['red', 'green'], display_str_list=["true", "pred"])
    plt.xlabel(n_predictions[i])
    plt.xticks([])
    plt.yticks([])
    
    if n_predictions[i] != n_labels[i]:
      ax.xaxis.label.set_color('red')
    
    plt.imshow(img_to_draw)

    if len(iou) > i :
      color = "black"
      if (n_iou[i][0] < iou_threshold):
        color = "red"
      ax.text(0.2, -0.3, "iou: %s" %(n_iou[i][0]), color=color, transform=ax.transAxes)
################################################################################

################################################################################
def dataset_to_numpy_util(training_dataset, validation_dataset, N, S):
  """
  Pull a batch from the datasets. This code is not very nice.
  
  Args:
    training_dataset : torch.utils.data.Dataset
        Dataset from the torch.utils.data.Dataset Pytorch class, returning the 
        training digits, labels and bboxes coordinates as batches such as : 
            - training digits : torch.Tensor of shape (batch_size, 1, 75, 75)
            - labels : torch.Tensor of shape (batch_size, 10)
            - bboxes coordinates : torch.Tensor of shape (batch_size, 4)
    validation_dataset : torch.utils.data.Dataset 
        Dataset from the torch.utils.data.Dataset Pytorch class, returning the 
        whole validation digits, labels and bboxes coordinates.
    N : int
        Size of the training sample to extract from the training dataset
    S : int
        Number of cell in one direction

  Returns:
    N_train_ds_digits : np.ndarray of shape (N, 1, 75, 75)
    N_train_ds_labels : np.ndarray of shape (N, 10)
    N_train_ds_bboxes : np.ndarray of shape (N, 4)
    validation_digits : np.ndarray of shape (len(validation_dataset), 1, 75, 75)
    validation_labels : np.ndarray of shape (len(validation_dataset), 10)
    validation_bboxes : np.ndarray of shape (len(validation_dataset), 4)
  """
  ### get N training digits, labels and bboxes from one batch
  ### turning the bboxes coordinates into ndarrays
  one_batch_train_ds_digits, one_batch_train_ds_labels, one_batch_train_ds_bboxes = next(iter(training_dataset))
  cell_size = 75/6 #one_batch_train_ds_digits[0][0].shape[1]/ S

  N_train_ds_digits = one_batch_train_ds_digits[:N].numpy()
  N_train_ds_labels = one_batch_train_ds_labels[:N].numpy()
  
  N_train_ds_bboxes = one_batch_train_ds_bboxes[:N].numpy()
  N_train_ds_bboxes_abs = np.zeros((N,4))


  x_min = ((N_train_ds_bboxes[:,2])) - (N_train_ds_bboxes[:,4]/2)
  y_min = ((N_train_ds_bboxes[:,3])) - (N_train_ds_bboxes[:,5]/2)
  x_max = ((N_train_ds_bboxes[:,2])) + (N_train_ds_bboxes[:,4]/2)
  y_max = ((N_train_ds_bboxes[:,3])) + (N_train_ds_bboxes[:,5]/2)


  N_train_ds_bboxes_abs[:,0] = x_min
  N_train_ds_bboxes_abs[:,1] = y_min
  N_train_ds_bboxes_abs[:,2] = x_max
  N_train_ds_bboxes_abs[:,3] = y_max


  print(N_train_ds_bboxes_abs[5])
  
  ### get the whole validation digits, labels and bboxes
  ### turning the bboxes coordinates into ndarrays
  for validation_digits, validation_labels, validation_bboxes in validation_dataset:
      validation_digits = validation_digits.numpy()
      validation_labels = validation_labels.numpy()
      validation_bboxes = validation_bboxes.numpy()
      break

  # turning one hot encoding labels into the corresponding digit
  validation_labels = np.argmax(validation_labels, axis=1)
  N_train_ds_labels = np.argmax(N_train_ds_labels, axis=1)

  return (N_train_ds_digits, N_train_ds_labels, N_train_ds_bboxes_abs,
          validation_digits, validation_labels, validation_bboxes)

In [5]:
### Choosing device between CPU or GPU
#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device = torch.device('mps') if torch.has_mps else torch.device('cpu')
print("\n------------------------------------")
print(f"Execute notebook on - {device} -")
print("------------------------------------\n")


------------------------------------
Execute notebook on - cpu -
------------------------------------



In [6]:
def numpy_pad_to_bounding_box(image, offset_height=0, offset_width=0, target_height=0, target_width=0):
    assert image.shape[:-1][0] <= target_height-offset_height, "height must be <= target - offset"
    assert image.shape[:-1][1] <= target_width-offset_width, "width must be <= target - offset"
    
    target_array = np.zeros((target_height, target_width, image.shape[-1]))

    for k in range(image.shape[0]):
        target_array[offset_height+k][offset_width:image.shape[1]+offset_width] = image[k]
    
    return target_array

In [7]:
class my_mnist_dataset(torch.utils.data.Dataset):
    def __init__(self, root:str, split:str=None, download:bool=False, S=6):
        assert split, "You have to specify the split."
        
        if split == "train":
            train = True
        elif split == "test":
            train = False
        
        self.dataset = torchvision.datasets.MNIST(root=root, train=train, download=download)
        
        self.cell_size = 75 / S
    
    def __len__(self):
        return len(self.dataset)

    def _transform_pasting75(self, image, label):
        ### xmin, ymin of digit
        xmin = torch.randint(0, 48, (1,))
        ymin = torch.randint(0, 48, (1,))
        
        image = torchvision.transforms.ToTensor()(image)
        image = torch.reshape(image, (28,28,1,))
        image = torch.from_numpy(numpy_pad_to_bounding_box(image, ymin, xmin, 75, 75))
        image = image.permute(2, 0, 1) #(C,H,W)
        image = image.to(torch.float)
        
        xmin, ymin = xmin.to(torch.float), ymin.to(torch.float)

        xmax_bbox, ymax_bbox = (xmin + 28), (ymin + 28)
        xmin_bbox, ymin_bbox = xmin, ymin
        w_bbox = xmax_bbox-xmin_bbox
        h_bbox = ymax_bbox-ymin_bbox

        rw = w_bbox / 75
        rh = h_bbox / 75
        cx = (xmin + (w_bbox/2))/75
        cy = (ymin + (h_bbox/2))/75

        cx_rcell = cx % self.cell_size / self.cell_size
        cy_rcell = cy % self.cell_size / self.cell_size


        label_one_hot = F.one_hot(torch.as_tensor(label, dtype=torch.int64), 10)
        bbox_coord = torch.Tensor([cx_rcell, cy_rcell, rw, rh])

        return image, label_one_hot, bbox_coord

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        image, one_hot_label, bbox_coord = self._transform_pasting75(self.dataset[idx][0], self.dataset[idx][1])
        
        return image, one_hot_label.to(torch.float), bbox_coord


In [8]:
def get_training_dataset(BATCH_SIZE=64):
    """
    Loads and maps the training split of the dataset using the custom dataset class. 
    """
    dataset = my_mnist_dataset(root="data", split="train", download=True)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
    return dataloader, len(dataset)

def get_validation_dataset(BATCH_SIZE = None):
    """
    Loads and maps the validation split of the datasetusing the custom dataset class. 
    """
    dataset = my_mnist_dataset(root="data", split="test", download=True)
    if BATCH_SIZE is None:
        BATCH_SIZE = len(dataset)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False)
    return dataloader, len(dataset)

# instantiate the datasets
training_dataset, len_training_ds = get_training_dataset()
validation_dataset, len_validation_ds = get_validation_dataset()

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [ ]:
(training_digits, training_labels, training_bboxes,
 validation_digits, validation_labels, validation_bboxes) = dataset_to_numpy_util(training_dataset, validation_dataset, 10, S=6)
display_digits_with_boxes(training_digits, training_labels, training_labels, np.array([]), training_bboxes, np.array([]), "training digits and their labels")

In [9]:
class CNNBlock(torch.nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super(CNNBlock, self).__init__()
        self.conv = torch.nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)
        self.bn = torch.nn.BatchNorm2d(out_channels)
        self.l_relu = torch.nn.LeakyReLU(0.1)
    
    def forward(self, input):
        x = self.conv(input)
        x = self.bn(x)
        return self.l_relu(x)

In [10]:
class YoloMNIST(torch.nn.Module):
    def __init__(self, sizeHW, S, C, B):
        super(YoloMNIST, self).__init__()
        self.S, self.C, self.B = S, C, B
        self.sizeHW = sizeHW
        self.cell_size = self.sizeHW / self.S

        self.seq = torch.nn.Sequential()        
        self.seq.add_module(f"conv_1", CNNBlock(1, 32, stride=2, kernel_size=7, padding=2))
        self.seq.add_module(f"maxpool_1", torch.nn.MaxPool2d(2))
        self.seq.add_module(f"conv_3", CNNBlock(32, 128, stride=1, kernel_size=3, padding=0))
        self.seq.add_module(f"maxpool_2", torch.nn.MaxPool2d(2))
        self.seq.add_module(f"conv_5", CNNBlock(128, 64, stride=1, kernel_size=1, padding=0))
        self.seq.add_module(f"conv_4", CNNBlock(64, 128, stride=1, kernel_size=3, padding=0))
        self.seq.add_module(f"conv_6", CNNBlock(128, 128, stride=1, kernel_size=3, padding=1))
        
        self.fcs = self._create_fcs()

    def _size_output(self, sizeHW:int, kernel:int, stride:int, padding:int=0, isMaxPool:bool=False)->int:
        """
        Output size (width/height) of convolutional or maxpool layers.

        Args:
            sizeHW : int
                Image size (we suppose this is a square image)
            kernel : int
                Size of a square kernel
            stride : int
                Stride of convolution layer
            padding : int
                Padding of convolution layer
            isMaxPool : Bool, default is False.
                Specify if it is a Maxpool layer (True) or not (False). 

        Return:
            output_size : int
                Image output size after a convolutional or MaxPool layer.
        """ 
        if isMaxPool == True:
            output_size = int(sizeHW/2)
            print(output_size)
            return output_size
        if padding == 'same':
            output_size = sizeHW
            print(output_size)
            return output_size
        else:
            output_size = (sizeHW + 2 * padding - (kernel-1)-1)/stride
            output_size = int(output_size + 1)
            print(output_size)
            return output_size

    def _create_fcs(self):
        output = torch.nn.Sequential(
            torch.nn.Flatten(),
            torch.nn.Linear(128 * self.S * self.S, 4096),
            torch.nn.LeakyReLU(0.1),
            torch.nn.Linear(4096, self.S * self.S * (self.C + self.B*5))
        )
        return output
    

    def forward(self, input:torch.Tensor)->tuple:
        """
        Forward pass.

        Args:
            input : torch.Tensor of shape (N, C, H, W)
                Batch of images.

        Return:
            box_coord : torch.Tensor of shape (N, 6, 6, 5)
                Contains xc_rcell, yc_rcell, rw, rh and the confidence number c
                over 6x6 grid cells.
            classifier : torch.Tensor of shape (N, 6, 6, 10)
                Contains the one-hot encoding of each digit number over
                6x6 grid cells.
        """     
        x = self.seq(input)
        x = self.fcs(x)
        x = x.view(x.size(0), self.S, self.S, self.B * 5 + self.C)
        box_coord = x[:,:,:,0:5]
        classifier = x[:,:,:,5:]
        return box_coord, classifier

In [ ]:
########### CELLULE TEST ##############################################
model_MNIST = YoloMNIST(sizeHW=75, S=6, C=10, B=1)
for k in training_dataset:
    break
print("### OUTPUT DATASET ###")
print("La taille d'un élément est de : ", len(k))
print("Le premier élément est l'image de shape : ", k[0].shape)
print("Le deuxieme élément est les label de shape : ", k[1].shape)
print("Le dernier élément est la bbox [xc_rcell, yc_rcell, xc, yc, rw, hw] : ", k[2].shape)
print("\n")

print("### OUTPUT PRED ###")
pc_pred, bbox_pred, class_pred = model_MNIST(k[0])
print("Le premier élément est une grille de shape : ", model_MNIST(k[0])[0].shape, " correspondant aux nombres pc sur les 6x6 cellules")
print("Le deuxieme élement est, une grille de shape : ", model_MNIST(k[0])[1].shape, " correspondant aux [xc_rcell, yc_rcell, rw, rh]")
print("Le troisieme élement est, une grille de shape : ", model_MNIST(k[0])[2].shape, " correspondant à des softmax vect sur chaque cellule de grille")
print("\n")

bbox_true = k[2][:1]
print(bbox_true.shape)
bbox_true2 = torch.concat((bbox_true[0][0:2], bbox_true[0][4::])).unsqueeze(0)
print(bbox_true2.shape)
print(k[2][0])
print(bbox_true2)
#######################################################################

In [11]:
class YoloLoss(torch.nn.Module):
    def __init__(self, lambd_coord:int, lambd_noobj:float, S:int, sizeHW:int):
        super(YoloLoss, self).__init__()
        self.lambd_coord = lambd_coord
        self.lambd_noobj = lambd_noobj
        self.S = S
        self.cell_size = sizeHW/S

    def _loss_over_coord(self, pred_coord_rcell, true_coord_rcell):
        """
        pred_coord_rcell : torch.Tensor of shape (N, 2)
        true_coord_rcell : torch.Tensor of shape (N, 2)
        """
        xc_hat, yc_hat = pred_coord_rcell
        xc, yc = true_coord_rcell
        squared_error = torch.pow(xc - xc_hat,2) - torch.pow(yc - yc_hat,2)
        return squared_error

    def _loss_over_size(self, pred_size, true_size):
        """
        pred_size : torch.Tensor of shape (N, 2)
        true_size : torch.Tensor of shape (N, 2)
        """
        rw_hat, rh_hat = pred_size
        rw, rh = true_size
        root_squared_error_w = torch.pow(torch.sqrt(rw) - torch.sqrt(rw_hat),2)
        root_squared_error_h = torch.pow(torch.sqrt(rw) - torch.sqrt(rw_hat),2)
        root_squared_error = root_squared_error_w - root_squared_error_h
        return root_squared_error

    def _loss_over_confidence(self, pred_c, c):
        """
        pred_c : torch.Tensor of shape (N, 1)
        c      : torch.Tensor of shape (1)
        """
        squared_error = torch.pow(c - pred_c,2)
        return squared_error

    def _loss_over_classprob(self, pred_class, true_class):
        """
        pred_class : torch.Tensor of shape (N, 10)
        true_class : torch.Tensor of shape (N, 10)
        """
        squared_error = torch.pow(true_class - pred_class,2)
        return squared_error
    
    def _relative2absolute(self, bbox_relative:torch.Tensor)->torch.Tensor:
        """
        Bounding box relative to cell coordinates into absolute coordinates 
        (pixels). Used to calculate IoU. 

        Args:
            bbox_relative : torch.Tensor of shape (N, 4)
                Bounding box coordinates to convert.
        Return:
            bbox_absolute : torch.Tensor of shape (N, 4)
        """
        cx_rcell, cy_rcell, rw, rh = bbox_relative.permute(1,0)
        
        cx = cx_rcell * self.cell_size - (1/self.cell_size) * int(cx_rcell/self.cell_size)
        cy = cy_rcell * self.cell_size - (1/self.cell_size) * int(cy_rcell/self.cell_size)

        cx_abs = self.sizeHW * cx
        cy_abs = self.sizeHW * cy
        x_min = cx_abs - (self.sizeHW * (rw/2))
        y_min = cy_abs - (self.sizeHW * (rh/2))
        x_max = cx_abs + (self.sizeHW * (rw/2))
        y_max = cy_abs + (self.sizeHW * (rh/2))

        bbox_absolute = torch.stack((x_min, y_min, x_max, y_max), dim=-1)
        return bbox_absolute

    def _intersection_over_union(self, pred_box:torch.Tensor, true_box:torch.Tensor)->float:
        """
        Intersection over Union method.

        Args:
            pred_box : torch.Tensor of shape (N, 4)
                Predicted bounding boxes of a batch, in a given cell.
            true_box : torch.Tensor of shape (N, 4)
                Ground truth bounding boxes of a batch, in a given cell.

        Return:
            iou : float
                Number between 0 and 1 where 1 is a perfect overlap.
        """        
        pred_box = self._relative2absolute(pred_box)
        true_box = self._relative2absolute(true_box)        
        xmin_pred, ymin_pred, xmax_pred, ymax_pred = pred_box.permute(1,0)
        xmin_true, ymin_true, xmax_true, ymax_true = true_box.permute(1,0)

        smoothing_factor = 1e-10

        xmin_overlap = torch.maximum(xmin_pred, xmin_true)
        xmax_overlap = torch.minimum(xmax_pred, xmax_true)
        ymin_overlap = torch.maximum(ymin_pred, ymin_true)
        ymax_overlap = torch.minimum(ymax_pred, ymax_true)
        
        pred_box_area = (xmax_pred - xmin_pred) * (ymax_pred - ymin_pred)
        true_box_area = (xmax_true - xmin_true) * (ymax_true - ymin_true)
       
        overlap_area = torch.maximum((xmax_overlap - xmin_overlap), torch.Tensor([0]))  * torch.maximum((ymax_overlap - ymin_overlap), torch.Tensor([0]))
        union_area = (pred_box_area + true_box_area) - overlap_area
        iou = (overlap_area + smoothing_factor) / (union_area + smoothing_factor)
        return iou   


    def forward(self, pred:torch.Tensor, truth:torch.Tensor, isObject:bool)->torch.Tensor:
        """
        Forward pass for the current grid cell.

        Args:
            pred : torch.Tensor of shape (N, 4+1+10)
                Batch predicted outputs containing xc_rcell, yc_rcell, rw, rh,
                confident number c and the one-hot vect of the digit class.
            truth : torch.Tensor of shape (N, 4+10) >>> 5+10 ???
                Groundtrue batch containing bbox values and digit class one-hot vect.
            isObject : bool
                Indicates if the center of the groundtrue bbox is in the current
                grid cell. In other words, indicates if there is an object to 
                detect in this grid cell.  

        Return:
            loss : float
                The loss value for the current grid cell
        """
        pred_box = pred[:4]
        true_box = truth[:4]
        
        pred_class = pred[5:]
        true_class = truth[4:]

        c = torch.Tensor([isObject])
        pred_c = pred[4] * self._intersection_over_union(pred_box, true_box)

        loss_1 = isObject * self._loss_over_coord(pred_box[:2], true_box[:2])
        loss_2 = isObject * self._loss_over_size(pred_box[2:], true_box[2:])
        loss_3 = isObject * self._loss_over_confidence(pred_c, c) # ?????
        loss_3b = (1-isObject) * self._loss_over_confidence(pred_c, c) ### ???  
        loss_4 = isObject * self._loss_over_classprob(pred_class, true_class)

        loss = self.lambd_coord*loss_1 + self.lambd_coord*loss_2 + loss_3 \
                + self.lambd_noobj * loss_3b + loss_4

        return loss/len(truth)

In [115]:
def bbox2Tensor(bbox:torch.Tensor, S:int, sizeHW:int)->torch.Tensor:
    """
    Constructs en Tensor and puts bbox values in the corresponding i,j grid cell.

    Args :
        bbox : torch.Tensor of shape (N,4)
            Contains bbox values xc_rcell, yc_rcell, rw and rh.
        S : int
            Size of the grid.
        sizeHW : int
            Size of the image.

    Return :
        bbox_t : torch.Tensor of shape (N, S, S, 4)
            Tensor containing all 4 bbox values in the corresponding i,j grid
            cell position i.e. in the i,j position where an object should be
            detected.
    """
    N = len(bbox)
    bbox_t = torch.zeros(N,S,S,4)
    gcell_size = 1/S
    cell_size = sizeHW/S

    xc_rcell, yc_rcell, rw, rh = bbox.permute(1,0)
    xc = xc_rcell * cell_size - (1/cell_size) * (xc_rcell/cell_size).to(torch.int32)
    yc = yc_rcell * cell_size - (1/cell_size) * (yc_rcell/cell_size).to(torch.int32)

    N_range = torch.arange(N)
    lines = (yc / gcell_size).to(torch.long)
    columns = (xc / gcell_size).to(torch.long)
    bbox_t[N_range, lines, columns] = torch.stack((xc_rcell, yc_rcell, rw, rh)).permute(1,0)
    
    return bbox_t

In [34]:
learning_rate = 0.001
BATCH_SIZE = 64

model_MNIST = YoloMNIST(sizeHW=75, S=6, C=10, B=1)
model_MNIST = model_MNIST.to(device)
optimizer = torch.optim.Adam(params=model_MNIST.parameters(), lr=learning_rate)
loss_yolo = YoloLoss(lambd_coord=5, lambd_noobj=0.5, S=6, sizeHW=75)
#isDigit_loss = torch.nn.MSELoss()
#regression_loss = nn.MSELoss()
#classification_loss = nn.CrossEntropyLoss()

print(optimizer)
summary(model_MNIST, input_size = (BATCH_SIZE,1,75,75))

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    eps: 1e-08
    foreach: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)


Layer (type:depth-idx)                   Output Shape              Param #
YoloMNIST                                [64, 6, 6, 5]             --
├─Sequential: 1-1                        [64, 128, 6, 6]           --
│    └─CNNBlock: 2-1                     [64, 32, 37, 37]          --
│    │    └─Conv2d: 3-1                  [64, 32, 37, 37]          1,568
│    │    └─BatchNorm2d: 3-2             [64, 32, 37, 37]          64
│    │    └─LeakyReLU: 3-3               [64, 32, 37, 37]          --
│    └─MaxPool2d: 2-2                    [64, 32, 18, 18]          --
│    └─CNNBlock: 2-3                     [64, 128, 16, 16]         --
│    │    └─Conv2d: 3-4                  [64, 128, 16, 16]         36,864
│    │    └─BatchNorm2d: 3-5             [64, 128, 16, 16]         256
│    │    └─LeakyReLU: 3-6               [64, 128, 16, 16]         --
│    └─MaxPool2d: 2-4                    [64, 128, 8, 8]           --
│    └─CNNBlock: 2-5                     [64, 64, 8, 8]            --
│    │ 

In [ ]:
######## TODO : réfléchir à comment utiliser bbox_tensor

In [ ]:
delta_time = datetime.timedelta(hours=1)
timezone = datetime.timezone(offset=delta_time)

t = datetime.datetime.now(tz=timezone)
str_t = '{:%Y-%m-%d %H:%M:%S}'.format(t)
print(f"[START] : {str_t} :")
print(f"[Training on] : {str(device).upper()}")

EPOCHS = 1
size_grid = 6
training_losses_list = []
box_losses_list = []
class_losses_list = []
pc_losses_list = []
model_MNIST = model_MNIST.to(device)

for epoch in range(EPOCHS) : 
    begin_time = timer()
    training_losses = 0
    
    print("-"*20)
    str_t = '{:%Y-%m-%d %H:%M:%S}'.format(t)
    print(" "*5 + f"{str_t} : EPOCH {epoch+1}/{EPOCHS}")
    print("-"*20)

    model_MNIST.train()
    for batch, (img, labels, bbox_true) in enumerate(training_dataset):
        begin_batch_time = timer()
        img, labels, bbox_true = img.to(device), labels.to(device), bbox_true.to(device)
        optimizer.zero_grad()
        
        bbox_coords_preds, label_preds = model_MNIST(img)

        loss = 0
        for i in range(size_grid):
            for j in range(size_grid):
                loss += loss_yolo(bbox_coords_preds, bbox_true)
        


        ### TODO 
        ### IoU over 6x6 cells
        ### Loss over 6x6 cells



        

        training_loss = box_loss + class_loss + pc_loss
        training_loss.backward()
        
        optimizer.step()
        
        ######### print part #######################
        training_losses += training_loss.item()
        training_losses_list.append(training_loss)
        box_losses_list.append(box_loss.item())
        class_losses_list.append(class_loss.item())
        pc_losses_list.append(pc_loss.item())

        if batch+1 <= len_training_ds//BATCH_SIZE:
            current_training_sample = (batch+1)*BATCH_SIZE
        else:
            current_training_sample = (batch)*BATCH_SIZE + len_training_ds%BATCH_SIZE
        
        if (batch+1) == 1 or (batch+1)%100 == 0 or (batch+1) == len_training_ds//BATCH_SIZE +1:
            print(f" --- Image : {current_training_sample}/{len_training_ds}",\
                  f" : loss = {training_loss:.5f}", \
                  f", classification loss = {class_loss.item():.5f}", \
                  f", bounding box loss = {box_loss.item():.5f}", \
                  f", pc loss = {pc_loss.item():.5f}")
            if batch+1 == (len_training_ds//BATCH_SIZE)+1:
                print(f"Total elapsed time for training : {datetime.timedelta(seconds=timer()-begin_time)}")
                print(f"Mean training loss for this epoch : {training_losses / len(training_dataset):.5f}")
    training_losses /= len(training_dataset)

[START] : 2022-08-14 19:44:19 :
[Training on] : MPS
--------------------
     2022-08-14 19:44:19 : EPOCH 1/1
--------------------


In [ ]:
447/6

74.5

In [ ]:
220%74.5/74.5

0.9530201342281879

In [ ]:
0.9530201342281879 * 74.5 + (2*74.5)

220.0